In [1]:
import pandas as pd
df = pd.read_csv("sampled_data_1233455.csv")

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
features = ["0","1", "2", "3", "4", "5", "7"]
X = df[features]
y = df["8"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(data = X_pca, columns = ["Feature 1", "Feature 2"])
dataset = pd.concat([pca_df, y], axis=1)
dataset=dataset.rename(columns={"8":"Class Label"})
dataset.head()

,Feature 1,Feature 2,Class Label
0,0.448372,-0.903387,0.0
1,-0.213044,0.550000,0.0
2,0.900868,-0.010139,1.0
3,0.374398,-0.850995,0.0
4,0.663910,-0.437368,0.0


In [3]:
dataset.shape

(1300, 3)

In [4]:
X = dataset[['Feature 1', 'Feature 2']].values
y = dataset['Class Label'].values

# Train-Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
from qiskit import Aer, QuantumCircuit, execute
from qiskit.circuit.library import ZZFeatureMap
import numpy as np
import pandas as pd

def quantum_kernel(X1, X2):
    backend = Aer.get_backend('statevector_simulator') #backend simulator

    feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement='linear') 

    #quantum circuits for each data point   
    circuits = [feature_map.bind_parameters(x) for x in np.vstack([X1, X2])]
    job = execute(circuits, backend)
    result = job.result()
    states = [result.get_statevector(circ) for circ in circuits]  
    states_X1 = states[:len(X1)]
    states_X2 = states[len(X1):]
    
    #absolute squared of the dot product 
    kernel_matrix = np.abs(np.dot(np.conj(states_X1), np.array(states_X2).T))**2
    
    return kernel_matrix


In [8]:
from sklearn.svm import SVC
clf = SVC(kernel=lambda X1, X2: quantum_kernel(X1, X2))
clf.fit(X_train, y_train)


SVC(kernel=<function <lambda> at 0x30d393560>)

In [9]:
train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)


In [10]:
from sklearn.metrics import classification_report

report_train = classification_report(y_train, train_predictions, target_names=['Class 0', 'Class 1'])
report_test = classification_report(y_test, test_predictions, target_names=['Class 0', 'Class 1'])

print("Classification Report for Training Data:")
print(report_train)
print("Classification Report for Testing Data:")
print(report_test)


Classification Report for Training Data:
              precision    recall  f1-score   support

     Class 0       0.65      0.50      0.57       532
     Class 1       0.58      0.71      0.64       508

    accuracy                           0.61      1040
   macro avg       0.61      0.61      0.60      1040
weighted avg       0.61      0.61      0.60      1040

Classification Report for Testing Data:
              precision    recall  f1-score   support

     Class 0       0.62      0.51      0.56       130
     Class 1       0.58      0.69      0.63       130

    accuracy                           0.60       260
   macro avg       0.60      0.60      0.60       260
weighted avg       0.60      0.60      0.60       260

